# Clustering Crypto

In [1]:
!pip install -U altair

     |████████████████████████████████| 727 kB 10.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
# Initial imports
import pandas as pd
#import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import altair as alt

### Data Preprocessing

In [5]:
# Loade the cryptocurrencies data
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [6]:
# Keep only cryptocurrencies that are on trading
crypto_df.drop(crypto_df[crypto_df["IsTrading"]==False].index, inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [7]:
# Keep only cryptocurrencies with a working algorithm
# ALL ARE DEFINED

In [8]:
# Remove the "IsTrading" column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [9]:
# Remove rows with at least 1 null value
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [10]:
crypto_df=crypto_df.dropna()

In [11]:
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [12]:
# Remove rows with cryptocurrencies without coins mined
file_path = Path("Resources/crypto_data_cleaned.csv")
crypto_df.to_csv(file_path, index=False)

#STILL 0's...

In [13]:
crypto_df = crypto_df[crypto_df.TotalCoinsMined != 0]

In [14]:
file_path = Path("Resources/crypto_data_cleaned2.csv")
crypto_df.to_csv(file_path, index=False)

#0's gone!

In [15]:
# Fetch the cryptocurrencies names prior to drop them from crypto_df
crypto_names = crypto_df['CoinName']
crypto_names.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [16]:
# Remove the cryptocurrency name since it's not going to be used on the clustering algorithm
crypto_df=crypto_df.drop(columns='CoinName', axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [17]:
# Create dummies variables for text features
dummies = pd.get_dummies(crypto_df[['Algorithm', 'ProofType']], drop_first=True)
X = pd.concat([crypto_df.drop(['Algorithm', 'ProofType'],axis=1), dummies],axis=1)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [19]:
# Use PCA to reduce dimension to 3 principal components
pca = PCA(n_components=3)
crypto_pca=pca.fit_transform(X_scaled)

In [20]:
# Create a DataFrame with the principal components data
pcs_df=pd.DataFrame(data=crypto_pca, columns=['principal component 1', 'principal component 2', 'principal component 3'], index=X.index)
pcs_df.head()

,principal component 1,principal component 2,principal component 3
42,-0.332211,1.010491,-0.494227
404,-0.315562,1.010659,-0.494471
1337,2.298656,1.674885,-0.696306
BTC,-0.145236,-1.330057,0.139177
ETH,-0.153481,-2.013130,0.386853


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [26]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km=KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data={'k': k, 'inertia': inertia}
df_elbow=pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia')

alt.Chart(...)

Running K-Means with `k=4`

In [27]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df['class'] = model.labels_

In [28]:
new_df=pd.concat([crypto_df, pcs_df], axis=1)
new_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.332211,1.010491,-0.494227,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315562,1.010659,-0.494471,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.298656,1.674885,-0.696306,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145236,-1.330057,0.139177,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153481,-2.013130,0.386853,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [38]:
# Create a 3D-Scatter with the PCA data and the clusters
alt.Chart(new_df).mark_circle(size=60).encode(
    x='principal component 1',
    y='principal component 2',
    color='class',
    tooltip=['TotalCoinsMined','TotalCoinSupply','Algorithm']
)

# alt.Chart(results).mark_circle(size=60).encode(
#     x='pc1',
#     y='pc2',
#     color='class',
#     tooltip=['Algorithm','TotalCoinsMined','TotalCoinSupply','CoinName']
# ).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [54]:
# Table with tradable cryptos
# new_df.hvplot.table(columns=['index', 'Algorithm', 'ProofType', 'TotalCoinSupply", "TotalCoinsMined', 'class'], width=400)

display(new_df)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class
42,Scrypt,PoW/PoS,0.005942,4.200000e-11,-0.332211,1.010491,-0.494227,0
404,Scrypt,PoW/PoS,0.007002,5.320000e-04,-0.315562,1.010659,-0.494471,0
1337,X13,PoW/PoS,0.035342,3.141593e-01,2.298656,1.674885,-0.696306,0
BTC,SHA-256,PoW,0.005960,2.100000e-05,-0.145236,-1.330057,0.139177,1
ETH,Ethash,PoW,0.006050,0.000000e+00,-0.153481,-2.013130,0.386853,1
...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,0.007951,2.000000e-03,2.491396,0.780645,-0.109099,0
GAP,Scrypt,PoW/PoS,0.005957,2.500000e-04,-0.330258,1.010380,-0.494240,0
BDX,CryptoNight,PoW,0.006927,1.400223e-03,0.327063,-2.297524,0.335646,1
ZEN,Equihash,PoW,0.005950,2.100000e-05,-0.155427,-2.056664,0.407961,1


In [40]:
# Print the total number of tradable cryptocurrencies
count= crypto_names.count()
print(f'There are {count} tradable cryptocurrencies')

There are 533 tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [41]:
# Scale data to create the scatter plot
data_scaler = MinMaxScaler()
new_df[['TotalCoinsMined', 'TotalCoinSupply']] = data_scaler.fit_transform(new_df[['TotalCoinsMined', 'TotalCoinSupply']])

In [44]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"


alt.Chart(new_df).mark_circle(size=60).encode(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    color='class',
    tooltip=['Algorithm','ProofType']
).interactive()

alt.Chart(...)